In [1]:
import plotly.io as pio
import plotly.graph_objects as go
import chart_studio.plotly as py
pio.renderers.default = "vscode"
import pandas as pd
import numpy as np
# print out all numpy array
np.set_printoptions(threshold=np.inf)

from scipy.interpolate import griddata

from itertools import product, combinations

from sklearn import datasets
from sklearn.gaussian_process import GaussianProcessRegressor

from collections import defaultdict

In [2]:
# load the dataset
data = pd.read_excel(
    "241010 Results.xlsx"
)
# !global prefix!
prefix = "241010"

In [3]:
header = data.columns
# slice the data, get rid of the first column
data = data.iloc[:, 1:]
data

,Temperature (°C),Rotation (RPM),Current (mA),Yield (Prod./IS)
0,20.000000,150.000000,20.000000,0.329519
1,60.000000,150.000000,20.000000,0.420495
2,20.000000,600.000000,20.000000,0.428062
3,60.000000,600.000000,20.000000,0.516990
4,20.000000,150.000000,100.000000,0.348837
5,60.000000,150.000000,100.000000,0.387779
6,20.000000,600.000000,100.000000,0.340672
7,20.000000,150.000000,20.000000,0.341067
8,60.000000,600.000000,100.000000,0.410377
9,40.000000,375.000000,60.000000,0.369296


In [4]:
# find the max value of the 'Yield (Prod./IS)' column, find the corresponding row index
max_yield = data['Yield (Prod./IS)'].max()
max_yield_index = data['Yield (Prod./IS)'].idxmax()
print(f"max_yield: {max_yield}, max_yield_index: {max_yield_index}")
print(f"condition of the max yield: \n{data.iloc[max_yield_index, 0:3]}")

max_yield: 0.6347169811320754, max_yield_index: 14
condition of the max yield: 
Temperature (°C)     73.635857
Rotation (RPM)      375.000000
Current (mA)         60.000000
Name: 14, dtype: float64


In [5]:
def save_fig(fig: go.Figure, filename: str, auto_open = True) -> None:
    pio.write_html(fig, file=filename, auto_open=auto_open)

In [6]:
def plot_original_data(
    data: pd.DataFrame,
    fig: go.Figure,
    axial_x: str,
    axial_y: str,
    axial_z: str,
    marker_size: int = 6,
) -> go.Figure:
    data_copy = data.copy()
    max_z_index = data[axial_z].idxmax()
    min_z_index = data[axial_z].idxmin()
    # drop the max and min value
    data_copy = data_copy.drop([max_z_index, min_z_index])
    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=data_copy[axial_x],
                y=data_copy[axial_y],
                z=data_copy[axial_z],
                mode="markers",
                marker=dict(
                    size=marker_size, color="blue", opacity=1.0, symbol="circle"
                ),
                name="Data Points",
            )
        ]
    )

    # Highlight the highest z axis value with red
    fig.add_trace(
        go.Scatter3d(
            x=[data[axial_x][max_z_index]],
            y=[data[axial_y][max_z_index]],
            z=[data[axial_z][max_z_index]],
            mode="markers",
            marker=dict(size=marker_size, color="red", opacity=1.0, symbol="diamond"),
            name="Highest Yield",
        )
    )

    # Highlight the lowest z axis value with green
    fig.add_trace(
        go.Scatter3d(
            x=[data[axial_x][min_z_index]],
            y=[data[axial_y][min_z_index]],
            z=[data[axial_z][min_z_index]],
            mode="markers",
            marker=dict(size=marker_size, color="green", opacity=1.0, symbol="square"),
            name="Lowest Yield",
        )
    )

    # Improve layout
    fig.update_layout(
        title=f"3D Scatter Plot of {axial_z} vs {axial_x} and {axial_y}",
        title_automargin=True,
        scene=dict(
            xaxis=dict(
                title=f"{axial_x}", showbackground=True, backgroundcolor="lightblue"
            ),
            yaxis=dict(
                title=f"{axial_y}", showbackground=True, backgroundcolor="lightcoral"
            ),
            zaxis=dict(
                title=f"{axial_z}",
                showbackground=True,
                backgroundcolor="lightgreen",
                nticks=10,
                range=[0, 100],
            ),
            aspectmode="cube",
        ),
        margin=dict(autoexpand=True, l=50, r=50, t=50, b=50),
        legend=dict(
            x=0.1,
            y=0.9,
            bgcolor="rgba(255, 255, 255, 0.5)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=16, weight=1000),
        ),
        template="plotly_white",
        meta=dict(
            data=data.to_json(), axial_x=axial_x, axial_y=axial_y, axial_z=axial_z
        ),
    )

    return fig

In [7]:
def plot_combinations(
    data: pd.DataFrame, first_3_columns: list, rest_columns: list
) -> None:
    for x, y in list(combinations(first_3_columns, 2)):
        for z in rest_columns:
            fig = plot_original_data(
                data=data,
                fig=go.Figure(),
                axial_x=x,
                axial_y=y,
                axial_z=z,
            )
            # save the plot
            fig_meta = fig["layout"]["meta"]
            axial_z = fig_meta["axial_z"].split(" ")[0]
            axial_x = fig_meta["axial_x"].split(" ")[0]
            axial_y = fig_meta["axial_y"].split(" ")[0]

            save_filename = (
                f"{prefix}_original_data_{axial_z}_vs_{axial_x}_and_{axial_y}.html"
            )
            print(f"Plotting {z} vs {x} and {y}, saving to {save_filename}")
            save_fig(
                fig,
                save_filename,
                auto_open=False,
            )

In [8]:
# plot all unique x, y combinations of the first 3 columns against the rest of the columns
# convert yield to percentage
data_copy = data.copy()
data_copy["Yield (Prod./IS)"] = data_copy["Yield (Prod./IS)"] * 100
plot_combinations(data_copy, data.columns[:3], data.columns[3:])

Plotting Yield (Prod./IS) vs Temperature (°C) and Rotation (RPM), saving to 241010_original_data_Yield_vs_Temperature_and_Rotation.html
Plotting Yield (Prod./IS) vs Temperature (°C) and Current (mA), saving to 241010_original_data_Yield_vs_Temperature_and_Current.html
Plotting Yield (Prod./IS) vs Rotation (RPM) and Current (mA), saving to 241010_original_data_Yield_vs_Rotation_and_Current.html


In [9]:
def bounds(data) -> list:
    return [[min(data[col]), max(data[col])] for col in data.columns]


def bounds_np(data) -> list:
    mins = np.min(data, axis=0)
    maxs = np.max(data, axis=0)
    return [[mins_, maxs_] for mins_, maxs_ in zip(mins, maxs)]


print(f"Using pandas: {bounds(data)}")
print(f"Using numpy: {bounds_np(data.to_numpy())}")

Using pandas: [[6.364143389851421, 73.63585661014858], [-3.403386864171523, 753.4033868641715], [20.0, 127.27171322029716], [0.3295194508009153, 0.6347169811320754]]
Using numpy: [[np.float64(6.364143389851421), np.float64(73.63585661014858)], [np.float64(-3.403386864171523), np.float64(753.4033868641715)], [np.float64(20.0), np.float64(127.27171322029716)], [np.float64(0.3295194508009153), np.float64(0.6347169811320754)]]


In [10]:
# create the line space
def fullfact_np(bound_array, num_levels: int) -> list:
    return np.array(list(product(*[np.linspace(min_, max_, num_levels) for min_, max_ in bound_array]))).tolist()

# create the line space
def fullfact(bound_array, num_levels: int) -> list:
    return np.array(list(product(*[np.linspace(min_, max_, num_levels) for min_, max_ in bound_array]))).tolist()

print(f"Using pandas: {fullfact(bounds(data.iloc[:, 0:3]), 2)}")
print(f"Using numpy: {fullfact_np(bounds_np(data.iloc[:, 0:3].to_numpy()), 2)}")

Using pandas: [[6.364143389851421, -3.403386864171523, 20.0], [6.364143389851421, -3.403386864171523, 127.27171322029716], [6.364143389851421, 753.4033868641715, 20.0], [6.364143389851421, 753.4033868641715, 127.27171322029716], [73.63585661014858, -3.403386864171523, 20.0], [73.63585661014858, -3.403386864171523, 127.27171322029716], [73.63585661014858, 753.4033868641715, 20.0], [73.63585661014858, 753.4033868641715, 127.27171322029716]]
Using numpy: [[6.364143389851421, -3.403386864171523, 20.0], [6.364143389851421, -3.403386864171523, 127.27171322029716], [6.364143389851421, 753.4033868641715, 20.0], [6.364143389851421, 753.4033868641715, 127.27171322029716], [73.63585661014858, -3.403386864171523, 20.0], [73.63585661014858, -3.403386864171523, 127.27171322029716], [73.63585661014858, 753.4033868641715, 20.0], [73.63585661014858, 753.4033868641715, 127.27171322029716]]


In [11]:
# preprocess the data, scale the data to -1 to 1
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
parameters_preprocessed = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
print(f"preprocessed data:")
display(parameters_preprocessed)

preprocessed data:


,Temperature (°C),Rotation (RPM),Current (mA),Yield (Prod./IS)
0,-0.891684,-0.891684,-1.094854,-0.963032
1,1.146451,-0.891684,-1.094854,0.236937
2,-0.891684,1.146451,-1.094854,0.336748
3,1.146451,1.146451,-1.094854,1.509720
4,-0.891684,-0.891684,1.139286,-0.708230
5,1.146451,-0.891684,1.139286,-0.194584
6,-0.891684,1.146451,1.139286,-0.815928
7,-0.891684,-0.891684,-1.094854,-0.810715
8,1.146451,1.146451,1.139286,0.103489
9,0.127383,0.127383,0.022216,-0.438382


In [12]:
# inverse the scaling
parameters_inverse = scaler.inverse_transform(parameters_preprocessed)
parameters_inverse = pd.DataFrame(parameters_inverse, columns=data.columns)
print(f"inverse the scaling:")
parameters_inverse

inverse the scaling:


,Temperature (°C),Rotation (RPM),Current (mA),Yield (Prod./IS)
0,20.000000,150.000000,20.000000,0.329519
1,60.000000,150.000000,20.000000,0.420495
2,20.000000,600.000000,20.000000,0.428062
3,60.000000,600.000000,20.000000,0.516990
4,20.000000,150.000000,100.000000,0.348837
5,60.000000,150.000000,100.000000,0.387779
6,20.000000,600.000000,100.000000,0.340672
7,20.000000,150.000000,20.000000,0.341067
8,60.000000,600.000000,100.000000,0.410377
9,40.000000,375.000000,60.000000,0.369296


In [13]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Define a kernel with different parameters
kernel = C(1.0, (1e-5, 1e5)) * RBF(10, (1e-3, 1e3))

# Create the Gaussian Process Regressor with a regularization term
regressor = GaussianProcessRegressor(alpha=1e-3, random_state=42)

# Fit the model
regressor.fit(parameters_preprocessed.iloc[:, :3], parameters_preprocessed.iloc[:, 3])

# Print the kernel and score
print(f"Kernel: {regressor.kernel_}")
print(f"Log Marginal Likelihood: {regressor.log_marginal_likelihood_value_}")

Kernel: 1**2 * RBF(length_scale=1)
Log Marginal Likelihood: -126.59275581401533


In [14]:
# predict the yield
def predict_yield(
    preprocessed_df: pd.DataFrame,
    regressor: GaussianProcessRegressor,
    scaler: StandardScaler,
    num_levels: int = 2,
) -> pd.DataFrame:
    # create the line space
    line_space = fullfact(bounds(preprocessed_df.iloc[:, 0:3]), num_levels)
    line_space_df = pd.DataFrame(line_space, columns=header[1:4])
    predicted_yield, std = regressor.predict(line_space_df.to_numpy(), return_std=True)

    # concatenate the predicted yield and the line space
    predicted_yield = np.concatenate(
        (line_space, predicted_yield.reshape(-1, 1)), axis=1
    )
    # inverse the scaling
    predicted_yield = scaler.inverse_transform(predicted_yield)
    predicted_yield = pd.DataFrame(predicted_yield, columns=preprocessed_df.columns)
    return predicted_yield, std

In [15]:
predicted_yield, std = predict_yield(
    preprocessed_df=parameters_preprocessed,
    regressor=regressor,
    scaler=scaler,
    num_levels=30,
)

c:\Users\R9000P\EChem_Auto_Potential_Optimization\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names



In [16]:
predicted_yield

,Temperature (°C),Rotation (RPM),Current (mA),Yield (Prod./IS)
0,6.364143,-3.403387,20.000000,0.375392
1,6.364143,-3.403387,23.699025,0.374811
2,6.364143,-3.403387,27.398049,0.374395
3,6.364143,-3.403387,31.097074,0.374134
4,6.364143,-3.403387,34.796098,0.374010
...,...,...,...,...
26995,73.635857,753.403387,112.475615,0.395843
26996,73.635857,753.403387,116.174639,0.393484
26997,73.635857,753.403387,119.873664,0.391755
26998,73.635857,753.403387,123.572689,0.390611


In [17]:
print(f"max predicted condition:")
display(predicted_yield.max())

max predicted condition:


Temperature (°C)     73.635857
Rotation (RPM)      753.403387
Current (mA)        127.271713
Yield (Prod./IS)      0.638981
dtype: float64

In [18]:
def plot_original_and_predicted_data(
    data: pd.DataFrame,
    predicted_data: dict,
    x_axis: str,
    y_axis: str,
    z_axis: str,
) -> go.Figure:
    # plot original data
    fig = plot_original_data(
        data=data,
        fig=go.Figure(),
        axial_x=x_axis,
        axial_y=y_axis,
        axial_z=z_axis,
    )

    # plot the predicted data
    # first find the max and min value of the predicted data, remove them
    overall_max_df = None
    overall_max_index = None
    overall_min_df = None
    overall_min_index = None
    # go through every key value pair in the predicted data, record the max and min index and corresponding data
    for key, value in predicted_data.items():
        df = value[0]
        current_max_index = df[z_axis].idxmax()
        current_min_index = df[z_axis].idxmin()
        if (
            overall_max_df is None
            or df[z_axis][current_max_index] > overall_max_df[z_axis][overall_max_index]
        ):
            overall_max_df = df
            overall_max_index = current_max_index
        if (
            overall_min_df is None
            or df[z_axis][current_min_index] < overall_min_df[z_axis][overall_min_index]
        ):
            overall_min_df = df
            overall_min_index = current_min_index
    # save the max and min value
    if overall_max_df is not None:
        overall_max_row = overall_max_df.iloc[overall_max_index]
        overall_max_df = overall_max_df.drop(overall_max_index)
        # plot the max value
        fig.add_trace(
            go.Scatter3d(
                x=[overall_max_row[x_axis]],
                y=[overall_max_row[y_axis]],
                z=[overall_max_row[z_axis]],
                mode="markers",
                marker=dict(size=6, color="gold", opacity=1.0, symbol="diamond"),
                name="Max Predicted Yield",
            )
        )
    if overall_min_df is not None:
        overall_min_row = overall_min_df.iloc[overall_min_index]
        overall_min_df = overall_min_df.drop(overall_min_index)
        fig.add_trace(
            go.Scatter3d(
                x=[overall_min_row[x_axis]],
                y=[overall_min_row[y_axis]],
                z=[overall_min_row[z_axis]],
                mode="markers",
                marker=dict(size=6, color="maroon", opacity=1.0, symbol="diamond"),
                name="Min Predicted Yield",
            )
        )

    # key is the name, value is the pandas dataframe for the data
    for key, value in predicted_data.items():
        df = value[0]
        fig.add_trace(
            go.Scatter3d(
                x=df[x_axis],
                y=df[y_axis],
                z=df[z_axis],
                mode="markers",
                marker=dict(size=3, color=value[1], opacity=0.5, symbol="circle"),
                name=f"{key}",
            )
        )

    # Improve layout
    fig.update_layout(
        title=f"{z_axis} vs {x_axis} and {y_axis} with Gaussian regression data",
        scene=dict(
            xaxis=dict(
                title=f"{x_axis}", showbackground=True, backgroundcolor="lightblue"
            ),
            yaxis=dict(
                title=f"{y_axis}", showbackground=True, backgroundcolor="lightcoral"
            ),
            zaxis=dict(
                title=f"{z_axis}",
                showbackground=True,
                backgroundcolor="lightgreen",
                nticks=10,
                range=[0, 100],
            ),
            aspectmode="cube",
        ),
        margin=dict(autoexpand=True, l=50, r=50, t=50, b=50),
        legend=dict(
            x=0.1,
            y=0.9,
            bgcolor="rgba(255, 255, 255, 0.5)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=16, weight=1000),
        ),
        template="plotly_white",
    )

    return fig

In [19]:
# convert predicted_yield to percentage
predicted_yield_copy = predicted_yield.copy()
predicted_yield_copy["Yield (Prod./IS)"] = (
    predicted_yield_copy["Yield (Prod./IS)"] * 100
)

# now do the same trick, plot every unique combination of the first 3 columns against the rest of the columns
for x, y in list(combinations(data.columns[:3], 2)):
    for z in data.columns[3:]:
        print(f"Plotting {z} vs {x} and {y} after Gaussian Regression")
        fig = plot_original_and_predicted_data(
            data=data_copy,
            predicted_data={
                "All Predicted Data Points": [predicted_yield_copy, "green"]
            },
            x_axis=x,
            y_axis=y,
            z_axis=z,
        )

        # save the plot
        fig_meta = fig["layout"]["meta"]
        axial_z = fig_meta["axial_z"].split(" ")[0]
        axial_x = fig_meta["axial_x"].split(" ")[0]
        axial_y = fig_meta["axial_y"].split(" ")[0]

        save_fig(
            fig,
            f"{prefix}_original_and_predicted_all_{axial_z}_vs_{axial_x}_and_{axial_y}.html",
        )

Plotting Yield (Prod./IS) vs Temperature (°C) and Rotation (RPM) after Gaussian Regression
Plotting Yield (Prod./IS) vs Temperature (°C) and Current (mA) after Gaussian Regression
Plotting Yield (Prod./IS) vs Rotation (RPM) and Current (mA) after Gaussian Regression


In [20]:
predicted_yield_copy.head()

,Temperature (°C),Rotation (RPM),Current (mA),Yield (Prod./IS)
0,6.364143,-3.403387,20.000000,37.539172
1,6.364143,-3.403387,23.699025,37.481095
2,6.364143,-3.403387,27.398049,37.439533
3,6.364143,-3.403387,31.097074,37.413367
4,6.364143,-3.403387,34.796098,37.400970


In [21]:
# generalize the function to filter either the max or min value
def filter_min_max(
    parameter: pd.DataFrame,
    predicted_yield: pd.Series,
    mode: str,
    x_axis_header: str,
    y_axis_header: str,
) -> pd.DataFrame:
    # Create a dictionary to store the highest yield for each (x, y) pair
    filtered_points = defaultdict(
        lambda: float("-inf") if mode == "max" else float("inf")
    )

    # Iterate over the points to keep the highest yield for each (x, y) pair
    for i in range(len(predicted_yield)):
        x, y, z = (
            parameter[x_axis_header][i],
            parameter[y_axis_header][i],
            predicted_yield[i],
        )
        if mode == "max":
            if z > filtered_points[(x, y)]:
                filtered_points[(x, y)] = z
        elif mode == "min":
            if z < filtered_points[(x, y)]:
                filtered_points[(x, y)] = z

    # Extract the filtered data
    filtered_x = []
    filtered_y = []
    filtered_z = []

    for (x, y), z in filtered_points.items():
        filtered_x.append(x)
        filtered_y.append(y)
        filtered_z.append(z)

    # generated header for the return dataframe, it will be the x, y, z columns corresponding header
    header_x = data.columns[data.columns.get_loc(x_axis_header)]
    header_y = data.columns[data.columns.get_loc(y_axis_header)]
    header_z = predicted_yield.name

    header = [header_x, header_y, header_z]

    return pd.DataFrame(list(zip(filtered_x, filtered_y, filtered_z)), columns=header)

In [22]:
# now do the same trick, plot every unique combination of the first 3 columns against the rest of the columns
for x, y in list(combinations(data.columns[:3], 2)):
    # pick these two columns out of the dataframes
    parameter = predicted_yield_copy[[x, y]]
    for z in data.columns[3:]:
        predicted_yield = predicted_yield_copy[z]
        print(
            f"Plotting {z} vs {x} and {y} after Gaussian Regression with only the max and min values"
        )
        filter_predicted_yield_max = filter_min_max(
            parameter=parameter,
            predicted_yield=predicted_yield,
            mode="max",
            x_axis_header=x,
            y_axis_header=y,
        )
        filter_predicted_yield_min = filter_min_max(
            parameter=parameter,
            predicted_yield=predicted_yield,
            mode="min",
            x_axis_header=x,
            y_axis_header=y,
        )

        fig = plot_original_and_predicted_data(
            data=data_copy,
            predicted_data={
                "Max Predicted Data Points": [filter_predicted_yield_max, "green"],
                "Min Predicted Data Points": [filter_predicted_yield_min, "red"],
            },
            x_axis=x,
            y_axis=y,
            z_axis=z,
        )

        # save the plot
        fig_meta = fig["layout"]["meta"]
        axial_z = fig_meta["axial_z"].split(" ")[0]
        axial_x = fig_meta["axial_x"].split(" ")[0]
        axial_y = fig_meta["axial_y"].split(" ")[0]

        save_fig(
            fig,
            f"{prefix}_original_and_predicted_only_max_and_min_{axial_z}_vs_{axial_x}_and_{axial_y}_.html",
        )

Plotting Yield (Prod./IS) vs Temperature (°C) and Rotation (RPM) after Gaussian Regression with only the max and min values
Plotting Yield (Prod./IS) vs Temperature (°C) and Current (mA) after Gaussian Regression with only the max and min values
Plotting Yield (Prod./IS) vs Rotation (RPM) and Current (mA) after Gaussian Regression with only the max and min values


In [23]:
for x, y in list(combinations(data.columns[:3], 2)):
    # pick these two columns out of the dataframes
    parameter = predicted_yield_copy[[x, y]]
    for z in data.columns[3:]:
        predicted_yield = predicted_yield_copy[z]
        print(f"Surface plot {z} vs {x} and {y} after Gaussian Regression")
        filter_predicted_yield_max = filter_min_max(
            parameter=parameter,
            predicted_yield=predicted_yield,
            mode="max",
            x_axis_header=x,
            y_axis_header=y,
        )
        # Create grid data for surface plot
        grid_x, grid_y = np.meshgrid(
            np.linspace(
                min(parameter[x]),
                max(parameter[x]),
                100,
            ),
            np.linspace(
                min(parameter[y]),
                max(parameter[y]),
                100,
            ),
        )
        # Interpolate the z values onto the grid
        grid_z = griddata(
            (filter_predicted_yield_max[x], filter_predicted_yield_max[y]),
            filter_predicted_yield_max[z],
            (grid_x, grid_y),
            method="cubic",
        )
        fig = plot_original_data(
            data=data_copy,
            fig=go.Figure(),
            axial_x=x,
            axial_y=y,
            axial_z=z,
        )
        # Create the surface plot
        fig.add_surface(
            x=grid_x,
            y=grid_y,
            z=grid_z,
            showscale=False,
            colorscale="Viridis",
        )
        # mark the max value
        max_index = filter_predicted_yield_max[z].idxmax()
        fig.add_scatter3d(
            x=[filter_predicted_yield_max[x][max_index]],
            y=[filter_predicted_yield_max[y][max_index]],
            z=[filter_predicted_yield_max[z][max_index]],
            mode="markers",
            marker=dict(size=8, color="gold", opacity=1.0, symbol="diamond"),
            name="Max Predicted Data Points",
        )
        # Improve layout
        fig.update_layout(
            title=f"{z} Surface Plot with Max Predicted Data Points",
            scene=dict(
                xaxis=dict(title=x, showbackground=True, backgroundcolor="lightblue"),
                yaxis=dict(title=y, showbackground=True, backgroundcolor="lightyellow"),
                zaxis=dict(
                    title=z,
                    showbackground=True,
                    backgroundcolor="lightgreen",
                    nticks=10,
                    range=[0, 100],
                ),
                aspectmode="cube",
            ),
            margin=dict(autoexpand=True, l=50, r=50, t=50, b=50),
            template="plotly_white",
            legend=dict(
                x=0.1,
                y=0.9,
                bgcolor="rgba(255, 255, 255, 0.5)",
                bordercolor="black",
                borderwidth=1,
                font=dict(size=16, weight=1000),
            ),
            meta=dict(
                axial_x=x,
                axial_y=y,
                axial_z=z,
            ),
        )

        # save the plot
        fig_meta = fig["layout"]["meta"]
        axial_z = fig_meta["axial_z"].split(" ")[0]
        axial_x = fig_meta["axial_x"].split(" ")[0]
        axial_y = fig_meta["axial_y"].split(" ")[0]

        # save the plot
        save_fig(
            fig,
            f"{prefix}_origin_surface_plot_{axial_z}_vs_{axial_x}_and_{axial_y}.html",
        )

Surface plot Yield (Prod./IS) vs Temperature (°C) and Rotation (RPM) after Gaussian Regression
Surface plot Yield (Prod./IS) vs Temperature (°C) and Current (mA) after Gaussian Regression
Surface plot Yield (Prod./IS) vs Rotation (RPM) and Current (mA) after Gaussian Regression
